In [1]:
#cd notebooks/YahooEmbeddings

In [1]:
import spacy
import pandas as pd
import numpy as np
from pathlib import Path
from models import DistributedBagOfWords,MeanPooling,PCA_Projection

In [5]:
# Setup the paths
cwd = Path.cwd()
data_path = cwd / 'data'
# Read the data
classes = pd.read_csv(data_path / 'classes.txt', header = None)
train = pd.read_csv(data_path / 'train.csv', header=None)
train.rename(columns={0: 'label', 1: 'question_title', 2: 'question_content', 3: 'best_answer'}, inplace=True)
train.fillna('', inplace=True)
test = pd.read_csv(data_path / 'test.csv', header=None)
test.fillna('', inplace=True)
test.rename(columns={0: 'label', 1: 'question_title', 2: 'question_content', 3: 'best_answer'}, inplace=True)
# Make the data into X and y
X_train = train.drop('label', axis=1)
y_train = train['label']
X_test = test.drop('label', axis=1)
y_test = test['label']

print(classes)

                        0
0       Society & Culture
1   Science & Mathematics
2                  Health
3   Education & Reference
4    Computers & Internet
5                  Sports
6      Business & Finance
7   Entertainment & Music
8  Family & Relationships
9   Politics & Government


In [11]:
# Concatenate the data to do a summary table of median words per class and data field
X = pd.concat([X_train, X_test], axis=0)
y = pd.concat([y_train, y_test], axis=0)
data = pd.concat([X, y], axis=1)
summary_data = data.assign(avg_title = data['question_title'].str.split().str.len(), 
                            avg_content = data['question_content'].str.split().str.len(),
                            avg_answer = data['best_answer'].str.split().str.len()).groupby('label').agg({'avg_title': 'median', 'avg_content': 'median', 'avg_answer': 'median'})
print(summary_data)

       avg_title  avg_content  avg_answer
label                                    
1           10.0         10.0        37.0
2            9.0          0.0        42.0
3           10.0          9.0        43.0
4            9.0          0.0        28.0
5           11.0          8.0        29.0
6           10.0          0.0        24.0
7           10.0          0.0        29.0
8            9.0          6.0        17.0
9           10.0         18.0        34.0
10          11.0          9.0        38.0


In [ ]:
# Perform the preprocessing of the data once

# Initialize the DBOW class and the MeanPool class that does preprocessing and vectorization
DBOW = DistributedBagOfWords()
MeanPool = MeanPooling()

# Run the DBOW on all the data and store it as a numpy array
#sum_vectors = DBOW.transform(X_train)
#np.save('sum_vectors.npy', sum_vectors)

# Run the MeanPooling on all the data and store it as a numpy array
#mean_vectors = MeanPool.transform(X_train)
#np.save('mean_vectors.npy', mean_vectors)